# Objetivo: ler e analisar as séries históricas baixadas <EM CONSTRUÇÃO>

Referência: https://github.com/Schiessl/AgoraVAI/blob/main/Importa_S%C3%A9rie_Hist%C3%B3rica_B3_Todas_as_a%C3%A7%C3%B5es_e_op%C3%A7%C3%B5es_v1_0_0.ipynb

### Importa bibliotecas

In [2]:
import pandas as pd
from datetime import datetime
import os
import zipfile

pd.set_option('display.max_columns', None)

### Layout do arquivo
Arquivo disponível em: https://www.b3.com.br/data/files/33/67/B9/50/D84057102C784E47AC094EA8/SeriesHistoricas_Layout.pdf

In [3]:
layout = [
    ("tipo_registro", 2, 'str'),
    ("data_pregao", 8, 'datetime'),
    ("cod_bdi", 2, 'str'),
    ("cod_negociacao", 12, 'str'),
    ("tipo_mercado", 3, 'str'),
    ("nome_empresa", 12, 'str'),
    ("especificacao_papel", 10, 'str'),
    ("prazo_dias_merc_termo", 3, 'str'),
    ("moeda_referencia", 4, 'str'),
    ("preco_abertura", 13, 'float'),
    ("preco_maximo", 13, 'float'),
    ("preco_minimo", 13, 'float'),
    ("preco_medio", 13, 'float'),
    ("preco_ultimo_negocio", 13, 'float'),
    ("preco_melhor_oferta_compra", 13, 'float'),
    ("preco_melhor_oferta_venda", 13, 'float'),
    ("numero_negocios", 5, 'int'),
    ("quantidade_papeis_negociados", 18, 'int'),
    ("volume_total_negociado", 18, 'float'),
    ("preco_exercicio", 13, 'float'),
    ("indicador_correcao_precos", 1, 'str'),
    ("data_vencimento", 8, 'datetime'),
    ("fator_cotacao", 7, 'int'),
    ("preco_exercicio_pontos", 13, 'float'),
    ("codigo_isin", 12, 'str'),
    ("num_distribuicao_papel", 3, 'str')
]

Alterei a função original para permitir a leitura do arquivo diretamente do ZIP

In [4]:
def importar_arquivo_b3(caminho_arquivo, nrows=None):
    """
    Função para importar dados de um arquivo no formato B3 (BM&FBOVESPA) e verificar a consistência.

    Parâmetros:
    - caminho_arquivo (str): O caminho do arquivo a ser importado.
    - nrows (int or None): Número de linhas a serem lidas do arquivo. Se None, lê todas as linhas.

    Retorna:
    - pd.DataFrame: Um DataFrame contendo os dados importados.

    A função lê o arquivo, realiza a importação para um DataFrame e verifica a consistência comparando a quantidade
    de registros importados com a quantidade informada no trailer.
    """
    inicio = datetime.now()
    print('Inicio da leitura:', inicio)

    # Definir os tipos de dados para cada coluna
    dtype_mapping = {
        field[0]: (
            'datetime64[ns]' if field[2] == 'datetime' else
            (float if field[2] == 'float' else
            (int if field[2] == 'int' else 'str'))
        ) for field in layout
    }

    largura_campo = [field[1] for field in layout]

    # Ler o arquivo diretamente com Pandas, lendo apenas as primeiras linhas se nrows for especificado
    df = pd.read_fwf(
        caminho_arquivo,
        compression='zip',
        widths=largura_campo,
        dtype=dtype_mapping,
        nrows=nrows,  # Número de linhas a serem lidas
        skipheader=1, # pula o header
        skipfooter=1 if nrows is None else 0 # pula o trailer quando lê arquivo completo
    )

    # Renomear as colunas
    df.columns = [field[0] for field in layout]

    # Converter a colunas com datas para datetime64[ns]
    df['data_pregao'] = pd.to_datetime(df['data_pregao'], format='%Y%m%d', errors='coerce')
    df['data_vencimento'] = pd.to_datetime(df['data_vencimento'], format='%Y%m%d', errors='coerce')
    # Arredondar campos para duas casas decimais
    decimal_fields = [field[0] for field in layout if field[2] == 'float']
    df[decimal_fields] = df[decimal_fields] / 100
    # Formatar o campo 'preco_exercicio_pontos' com 6 casas decimais
    df['preco_exercicio_pontos'] = df['preco_exercicio_pontos'] / 1e6
    # Aplicar formatação específica apenas para o campo volume_total_negociado
    df['volume_total_negociado'] = df['volume_total_negociado'].apply(lambda x: '{:.2f}'.format(x))

    # Exibir a quantidade de linhas importadas
    # print('Quantidade de linhas importadas:', len(df))

    if nrows is None:
        # Se lendo todas as linhas, verificar o trailer
        # Exibir a quantidade lida no trailer (posições 32 a 42)
        with zipfile.ZipFile(caminho_arquivo, 'r') as zip_ref:
            with zip_ref.open(os.path.basename(caminho_arquivo).replace('.ZIP', '.TXT'), 'r') as file:
                lines = file.readlines()
                trailer = lines[-1]
                quantidade_lida_no_trailer = int(trailer[32:42])-2  # exclui header e trailer

        if len(df) == quantidade_lida_no_trailer:
            print(f'\n#### Arquivo importado com sucesso! - {len(df):,.0f} registros em {(datetime.now() - inicio)} ####')
        else:
            print(f'\n---- Arquivo não foi importado corretamente! ----')
            print(f'---- Foram importadas {len(df):,.0f} linhas, mas o trailer indica {quantidade_lida_no_trailer} linhas. ----')
    else:
            print(f'\n---- Foram importadas {len(df):,.0f} linhas. ----')

    print('\nFim da leitura:', datetime.now())
    return df

In [5]:
# Exemplo da leitura de um arquivo
caminho_do_arquivo = 'C:/repos/b3/series_historicas/dados/avaliar/COTAHIST_A2020.ZIP'

df = importar_arquivo_b3(caminho_do_arquivo)

Inicio da leitura: 2024-08-10 11:39:36.181558

#### Arquivo importado com sucesso! - 1,251,646 registros em 0:00:22.894337 ####

Fim da leitura: 2024-08-10 11:39:59.076919


In [6]:
# Mostra aleatóriamente alguns registros
df.sample(5)

,tipo_registro,data_pregao,cod_bdi,cod_negociacao,tipo_mercado,nome_empresa,especificacao_papel,prazo_dias_merc_termo,moeda_referencia,preco_abertura,preco_maximo,preco_minimo,preco_medio,preco_ultimo_negocio,preco_melhor_oferta_compra,preco_melhor_oferta_venda,numero_negocios,quantidade_papeis_negociados,volume_total_negociado,preco_exercicio,indicador_correcao_precos,data_vencimento,fator_cotacao,preco_exercicio_pontos,codigo_isin,num_distribuicao_papel
387349,1,2020-02-17,78,BRMLC18,70,BRML FM,ON NM,0.0,R$,0.66,0.66,0.54,0.55,0.54,0.50,1.08,34,184600,102132.00,18.00,0,2020-03-16,1,0.0,BRBRMLACNOR9,115
783638,1,2020-07-29,78,CCROH180,70,CCRO,ON NM,0.0,R$,0.05,0.05,0.05,0.05,0.05,0.00,0.11,1,1000,50.00,18.00,0,2020-08-17,1,0.0,BRCCROACNOR2,137
994873,1,2020-10-13,82,JBSSW172,80,JBSSE,ON NM,0.0,R$,0.30,0.30,0.18,0.20,0.22,0.15,0.29,15,20600,4209.00,17.21,0,2020-11-16,1,0.0,BRJBSSACNOR8,115
1024902,1,2020-10-21,78,USIMK133,70,USIM,PNA N1,0.0,R$,0.10,0.16,0.09,0.13,0.15,0.14,0.15,152,366300,50133.00,13.31,0,2020-11-16,1,0.0,BRUSIMACNPA6,162
1170653,1,2020-12-07,82,CIELM350,80,CIELE FM,ON NM,0.0,R$,0.14,0.14,0.13,0.13,0.13,0.10,0.15,7,3000,394.00,3.50,0,2021-01-18,1,0.0,BRCIELACNOR3,142
